In [1]:
#import required libraries
import nfl_data_py as nfl
import pandas as pd

In [2]:
#load play-by-play data for multiple seasons
pbp_data = nfl.import_pbp_data(range(2019, 2024))

2019 done.
2020 done.
2021 done.
2022 done.
2023 done.
Downcasting floats.


In [3]:
#aggregate key stats per game per team 
game_stats = pbp_data.groupby(["game_id", "posteam"]).agg(
    #meta stats
    week=("week", "first"),
    season=("season", "first"),
    scores=("posteam_score", "max"),
    
    #offensive stats
    passing_yards=("passing_yards", "sum"),
    rushing_yards=("rushing_yards", "sum"),
    passing_touchdown=("pass_touchdown", "sum"),
    rushing_touchdown=("rush_touchdown", "sum"),
    highest_yac=("yards_after_catch", "max"),
    time_to_throw=("time_to_throw", "mean"),
    sacks_taken=("sack", "sum"),

    #efficiency stats
    fourth_down_converted=("fourth_down_converted", "sum"),
    third_down_converted=("third_down_converted", "sum"),
    
    #turnovers
    fumbles=("fumble_lost", "sum"),
    interceptions=("interception", "sum"),
    fumbles_forced=("fumble_forced", "sum"),
    incomplete_pass=("incomplete_pass", "sum"),

    #penalties
    penalties=("penalty", "sum"),
    penalty_yards=("penalty_yards", "sum"),
    
    #advanced metrics
    epa=("epa", "sum"), #expected points added
    pass_epa=("total_home_pass_epa", "sum"),
    rush_epa=("total_home_rush_epa", "sum"),
    wpa=("wpa", "sum"), #win probability added

    #defensive pressure
    qb_hits=("qb_hit", "sum"),
    tackles_for_loss=("tackled_for_loss", "sum"),

    #special teams
    punts_inside_20=("punt_inside_twenty", "sum"),
    punt_attempts=("punt_attempt", "sum"),
    punts_blocked=("punt_blocked", "sum"),
    return_yards=("return_yards", "sum"),

    #drive & series stats
    drives=("drive", "max"),
    longest_drive=("drive_play_count", "max")
).reset_index()

#rename possession team variable name for clarity
game_stats.rename(columns={"posteam": "team"}, inplace=True)
game_stats.head()

,game_id,team,week,season,scores,passing_yards,rushing_yards,passing_touchdown,rushing_touchdown,highest_yac,...,rush_epa,wpa,qb_hits,tackles_for_loss,punts_inside_20,punt_attempts,punts_blocked,return_yards,drives,longest_drive
0,2019_01_ATL_MIN,ATL,1,2019,12.0,304.0,73.0,2.0,0.0,16.0,...,766.721191,-0.334475,7.0,3.0,2.0,4.0,1.0,44.0,19.0,13.0
1,2019_01_ATL_MIN,MIN,1,2019,28.0,98.0,172.0,1.0,3.0,12.0,...,672.384460,0.098436,2.0,3.0,2.0,5.0,0.0,59.0,20.0,10.0
2,2019_01_BAL_MIA,BAL,1,2019,59.0,379.0,265.0,6.0,2.0,43.0,...,-1138.722168,0.456696,1.0,3.0,1.0,1.0,0.0,9.0,22.0,14.0
3,2019_01_BAL_MIA,MIA,1,2019,10.0,190.0,21.0,1.0,0.0,18.0,...,-850.117737,-0.114414,12.0,4.0,2.0,6.0,0.0,102.0,21.0,8.0
4,2019_01_BUF_NYJ,BUF,1,2019,17.0,254.0,128.0,1.0,1.0,14.0,...,24.533037,-0.155580,9.0,2.0,1.0,3.0,0.0,21.0,22.0,9.0


In [4]:
#aggregate game outcomes
wins = pbp_data.groupby("game_id").agg(
    home_team=("home_team", "first"),
    away_team=("away_team", "first"),
    home_score=("total_home_score", "max"),
    away_score=("total_away_score", "max")
).reset_index()

#create win/loss columns
wins["home_win"] = (wins["home_score"] > wins["away_score"]).astype(int)
wins["away_win"] = (wins["away_score"] > wins["home_score"]).astype(int)

#reshape data so each row represents one team's result per game 
#and standardize column names for merging
home_wins = wins[["game_id", "home_team", "home_win"]].rename(
    columns={"home_team": "team", "home_win": "win"})
away_wins = wins[["game_id", "away_team", "away_win"]].rename(
    columns={"away_team": "team", "away_win": "win"})

#combine home and away results
game_results = pd.concat([home_wins, away_wins])

#merge win/loss data with team stats
team_game_stats = game_stats.merge(game_results, on=["game_id", "team"])
team_game_stats.head()

,game_id,team,week,season,scores,passing_yards,rushing_yards,passing_touchdown,rushing_touchdown,highest_yac,...,wpa,qb_hits,tackles_for_loss,punts_inside_20,punt_attempts,punts_blocked,return_yards,drives,longest_drive,win
0,2019_01_ATL_MIN,ATL,1,2019,12.0,304.0,73.0,2.0,0.0,16.0,...,-0.334475,7.0,3.0,2.0,4.0,1.0,44.0,19.0,13.0,0
1,2019_01_ATL_MIN,MIN,1,2019,28.0,98.0,172.0,1.0,3.0,12.0,...,0.098436,2.0,3.0,2.0,5.0,0.0,59.0,20.0,10.0,1
2,2019_01_BAL_MIA,BAL,1,2019,59.0,379.0,265.0,6.0,2.0,43.0,...,0.456696,1.0,3.0,1.0,1.0,0.0,9.0,22.0,14.0,1
3,2019_01_BAL_MIA,MIA,1,2019,10.0,190.0,21.0,1.0,0.0,18.0,...,-0.114414,12.0,4.0,2.0,6.0,0.0,102.0,21.0,8.0,0
4,2019_01_BUF_NYJ,BUF,1,2019,17.0,254.0,128.0,1.0,1.0,14.0,...,-0.155580,9.0,2.0,1.0,3.0,0.0,21.0,22.0,9.0,1


In [5]:
#filter for stats columns
stat_cols = team_game_stats.columns[4:-1]

#sort by team, season, and week to ensure past games are in order
team_game_stats = team_game_stats.sort_values(["team", "season", "week"]).reset_index(drop=True)

#compute rolling averages of the last 5 games for each team within each season.
rolling_avg = team_game_stats.copy()
rolling_avg[stat_cols] = team_game_stats.groupby(["team", "season"])[stat_cols].shift(1).rolling(window=5, 
                                                                                           min_periods=5).mean()

#remove first 5 games per team and season since they don't have a full rolling average
rolling_avg = rolling_avg.groupby(["team", "season"], group_keys=False).apply(lambda x: x.iloc[5:]).reset_index(drop=True)
rolling_avg.head()

,game_id,team,week,season,scores,passing_yards,rushing_yards,passing_touchdown,rushing_touchdown,highest_yac,...,wpa,qb_hits,tackles_for_loss,punts_inside_20,punt_attempts,punts_blocked,return_yards,drives,longest_drive,win
0,2019_06_ATL_ARI,ARI,6,2019,19.4,264.8,126.8,0.8,0.8,19.0,...,0.211505,6.2,0.8,1.6,4.8,0.0,63.4,21.6,12.4,1
1,2019_07_ARI_NYG,ARI,7,2019,20.8,271.2,124.8,1.0,1.0,23.4,...,0.297419,5.0,1.0,1.4,3.8,0.0,62.6,19.6,13.4,1
2,2019_08_ARI_NO,ARI,8,2019,22.8,222.2,152.0,1.0,1.4,20.2,...,0.317910,3.6,1.8,1.4,3.8,0.2,45.2,20.2,13.4,0
3,2019_09_SF_ARI,ARI,9,2019,20.6,231.6,135.8,0.6,1.4,20.0,...,0.227159,2.2,1.8,1.2,3.8,0.2,47.6,19.8,12.2,0
4,2019_10_ARI_TB,ARI,10,2019,23.2,231.6,143.4,1.0,1.4,27.0,...,0.319941,1.8,2.2,1.0,4.2,0.2,58.8,19.8,12.4,0


In [6]:
#sort by game_id to ensure game matchups are in order
rolling_avg = rolling_avg.sort_values(["game_id"]).reset_index(drop=True)

#compute the difference in stats between the two teams in each game
game_differentials = rolling_avg.copy()
game_differentials[stat_cols] = (
    rolling_avg.groupby("game_id")[stat_cols].diff().fillna(0) + #team 2 - team 1
    rolling_avg.groupby("game_id")[stat_cols].diff(-1).fillna(0) #team 1 - team 2
)
game_differentials.head()

,game_id,team,week,season,scores,passing_yards,rushing_yards,passing_touchdown,rushing_touchdown,highest_yac,...,wpa,qb_hits,tackles_for_loss,punts_inside_20,punt_attempts,punts_blocked,return_yards,drives,longest_drive,win
0,2019_06_ATL_ARI,ARI,6,2019,-0.8,-66.2,59.2,-1.4,0.4,-4.0,...,0.183276,-1.0,-1.8,0.6,1.6,-0.2,-0.6,1.6,-0.4,1
1,2019_06_ATL_ARI,ATL,6,2019,0.8,66.2,-59.2,1.4,-0.4,4.0,...,-0.183276,1.0,1.8,-0.6,-1.6,0.2,0.6,-1.6,0.4,0
2,2019_06_CAR_TB,CAR,6,2019,-4.0,-25.0,34.2,-1.2,0.8,-2.2,...,0.219131,-2.8,-0.6,0.8,0.2,0.0,4.4,1.2,-2.2,1
3,2019_06_CAR_TB,TB,6,2019,4.0,25.0,-34.2,1.2,-0.8,2.2,...,-0.219131,2.8,0.6,-0.8,-0.2,0.0,-4.4,-1.2,2.2,0
4,2019_06_CIN_BAL,BAL,6,2019,15.6,-17.2,131.0,1.0,1.2,-7.0,...,0.226656,-1.6,-0.4,0.4,-2.0,0.0,-8.6,-0.4,1.2,1


In [7]:
#export data to csv file for modeling
game_differentials.to_csv("rolling_average_differentials.csv", index=False)
# rolling_avg.to_csv("rolling_averages.csv", index=False)

### In the future can add in player injuries and weather data:
    nfl.import_injuries(range(2022,2024))[nfl.import_injuries(range(2022,2024))["full_name"] == "T.J. Watt"]